In [2]:
!apt-get install ffmpeg freeglut3-dev xvfb  # For visualization
!pip install "stable-baselines3[extra]>=2.0.0a4"
!pip install tensorboard
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import gym
from torch.distributions import Categorical

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
The following additional packages will be installed:
  freeglut3 libegl-dev libfontenc1 libgl-dev libgl1-mesa-dev libgles-dev libgles1 libglu1-mesa
  libglu1-mesa-dev libglvnd-core-dev libglvnd-dev libglx-dev libice-dev libopengl-dev libsm-dev
  libxfont2 libxkbfile1 libxt-dev x11-xkb-utils xfonts-base xfonts-encodings xfonts-utils
  xserver-common
Suggested packages:
  libice-doc libsm-doc libxt-doc
The following NEW packages will be installed:
  freeglut3 freeglut3-dev libegl-dev libfontenc1 libgl-dev libgl1-mesa-dev libgles-dev libgles1
  libglu1-mesa libglu1-mesa-dev libglvnd-core-dev libglvnd-dev libglx-dev libice-dev libopengl-dev
  libsm-dev libxfont2 libxkbfile1 libxt-dev x11-xkb-utils xfonts-base xfonts-encodings xfonts-utils
  xserver-common xvfb
0 upgraded, 25 newly installed, 0 to remove and 49 not upgraded.
Need t

In [17]:

# Define the policy network
class PolicyNetwork(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(PolicyNetwork, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.fc2 = nn.Linear(128, output_dim)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.softmax(self.fc2(x), dim=-1)
        return x

# Sample action from the policy network
def select_action(policy, state):
    state = torch.tensor(state, dtype=torch.float32)
    probs = policy(state)
    dist = torch.distributions.Categorical(probs)
    action = dist.sample()
    return action.item(), dist.log_prob(action)

# HARPG Algorithm
def harpg(env, policy, optimizer, T, eta, gamma):
    theta_t = torch.nn.utils.parameters_to_vector(policy.parameters())
    theta_t_prev = theta_t.clone()
    d_t = torch.zeros_like(theta_t)

    for t in range(T - 1):
        # Reset the environment and get the initial state
        obs, _ = env.reset()  # Adjust for Gym version 0.26+
        state = obs  # Use the first element as the observation

        done = False
        episode_reward = 0

        # Run an episode and collect gradients
        log_probs = []
        rewards = []
        while not done:
            action, log_prob = select_action(policy, state)

            # env.step() now returns next_state, reward, done, and info
            next_state, reward, done, _ = env.reset(action)

            log_probs.append(log_prob)
            rewards.append(reward)
            state = next_state
            episode_reward += reward

        # Compute returns
        returns = []
        G = 0# Define the policy network
class PolicyNetwork(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(PolicyNetwork, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.fc2 = nn.Linear(128, output_dim)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.softmax(self.fc2(x), dim=-1)
        return x

# Sample action from the policy network
def select_action(policy, state):
    state = torch.tensor(state, dtype=torch.float32)
    probs = policy(state)
    dist = torch.distributions.Categorical(probs)
    action = dist.sample()
    return action.item(), dist.log_prob(action)

# HARPG Algorithm
def harpg(env, policy, optimizer, T, eta, gamma):
    theta_t = torch.nn.utils.parameters_to_vector(policy.parameters())
    theta_t_prev = theta_t.clone()
    d_t = torch.zeros_like(theta_t)

    all_rewards = []  # To store rewards for each episode

    for t in range(T - 1):
        # Reset the environment and get the initial state
        obs = env.reset()  # Adjust for Gym version 0.26+
        state = obs  # Use the first element as the observation

        done = False
        episode_reward = 0

        # Run an episode and collect gradients
        log_probs = []
        rewards = []
        while not done:
            action, log_prob = select_action(policy, state)

            # env.step() now returns next_state, reward, done, and info
            next_state, reward, done, _ = env.step(action)

            log_probs.append(log_prob)
            rewards.append(reward)
            state = next_state
            episode_reward += reward

        # Print rewards for the current episode
        print(f"Episode {t+1}: Reward = {episode_reward}")
        all_rewards.append(episode_reward)

        # Compute returns
        returns = []
        G = 0
        for r in reversed(rewards):
            G = r + gamma * G
            returns.insert(0, G)

        returns = torch.tensor(returns, dtype=torch.float32)

        # Compute gradient g(τ_t, θ_t)
        loss = -torch.sum(torch.stack(log_probs) * returns)
        optimizer.zero_grad()
        loss.backward()

        g_t = torch.nn.utils.parameters_to_vector([p.grad for p in policy.parameters()])
        q_t = np.random.uniform()

        if g_t is None:
            raise ValueError("g_t is None.")

        # θ̂_t = q_tθ_t + (1 − q_t)θ_{t−1}
        theta_hat_t = q_t * theta_t + (1 - q_t) * theta_t_prev

        # Hessian-Aided update
        v_t = theta_hat_t - theta_t_prev
        d_t = (1 - eta) * (d_t + v_t) + eta * g_t

        # Update parameters θ_{t+1} = θ_t + γ_t d_t
        theta_t_prev = theta_t.clone()
        theta_t = theta_t + gamma * d_t

        # Update the policy network weights
        torch.nn.utils.vector_to_parameters(theta_t, policy.parameters())

    return theta_t, all_rewards  # Return final theta and all rewards

env = gym.make("CartPole-v1")
gamma = 0.99
input_dim = env.observation_space.shape[0]
output_dim = env.action_space.n
policy = PolicyNetwork(input_dim, output_dim)
optimizer = optim.Adam(policy.parameters(), lr=1e-2)
# Example usage:
final_theta, rewards = harpg(env, policy, optimizer, T=100, eta=0.1, gamma=0.99)
#print("All rewards across episodes:", rewards)

G=0
returns=[]
for r in reversed(rewards):
      G = r + gamma * G
      returns.insert(0, G)

returns = torch


Episode 1: Reward = 10.0
Episode 2: Reward = 10.0
Episode 3: Reward = 10.0
Episode 4: Reward = 8.0
Episode 5: Reward = 8.0
Episode 6: Reward = 9.0
Episode 7: Reward = 9.0
Episode 8: Reward = 8.0
Episode 9: Reward = 9.0
Episode 10: Reward = 9.0
Episode 11: Reward = 10.0
Episode 12: Reward = 8.0
Episode 13: Reward = 9.0
Episode 14: Reward = 9.0
Episode 15: Reward = 8.0
Episode 16: Reward = 10.0
Episode 17: Reward = 9.0
Episode 18: Reward = 10.0
Episode 19: Reward = 9.0
Episode 20: Reward = 9.0
Episode 21: Reward = 9.0
Episode 22: Reward = 10.0
Episode 23: Reward = 8.0
Episode 24: Reward = 10.0
Episode 25: Reward = 10.0
Episode 26: Reward = 8.0
Episode 27: Reward = 9.0
Episode 28: Reward = 9.0
Episode 29: Reward = 9.0
Episode 30: Reward = 10.0
Episode 31: Reward = 9.0
Episode 32: Reward = 10.0
Episode 33: Reward = 8.0
Episode 34: Reward = 8.0
Episode 35: Reward = 10.0
Episode 36: Reward = 9.0
Episode 37: Reward = 9.0
Episode 38: Reward = 11.0
Episode 39: Reward = 10.0
Episode 40: Reward =